In [1]:
import pandas as pd
import random
import numpy as np
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,make_scorer,log_loss,roc_curve,precision_recall_curve
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
pwd

'/Users/janmichaelaustria/Google Drive/Insight 2020/Sepsis Detection/Version3_Modeling_XGb'

In [3]:
file = '/Users/janmichaelaustria/Google Drive/Insight 2020/Sepsis Detection/24hr_windows_sep_only.csv'
sepsis = pd.read_csv(file,index_col=0)

/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
X = sepsis.iloc[:,3:-2]
y = sepsis['SepsisLabel']
del sepsis

In [5]:
X.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,27,1,0.0,0.0,0.03,1
1,117.0,99.0,0.0,116.0,97.0,81.0,20.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,27,1,0.0,0.0,0.03,2
2,117.6,99.2,0.0,116.4,94.4,77.6,22.0,0.0,-7.0,23.0,0.0,7.15,66.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.0,3.6,0.0,0.0,4.0,0.0,0.0,37.6,12.70,46.4,8.900000,124.000000,64.000000,27,1,0.0,0.0,0.03,3
3,118.2,99.4,0.0,116.8,91.8,74.2,24.0,0.0,-4.0,23.5,0.0,7.13,80.5,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.5,2.7,0.0,0.0,3.6,0.0,0.0,31.0,9.65,40.8,3.900000,180.000000,114.000000,27,1,0.0,0.0,0.03,4
4,118.8,99.6,0.0,117.2,89.2,70.8,26.0,0.0,-1.0,24.0,0.0,7.30,52.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102.0,3.8,0.0,0.0,3.6,0.0,0.0,28.0,9.30,39.8,4.033333,190.666667,115.333333,27,1,0.0,0.0,0.03,5


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True)

In [ ]:
xgb_model = xgb.XGBClassifier()
parameters = {'objective':['binary:logistic'],
              'max_depth': [9],
              'n_estimators': [500]}
scoring = {'f1': make_scorer(f1_score), 
           'Accuracy': make_scorer(accuracy_score),
          'precision': make_scorer(precision_score),
          'recall':make_scorer(recall_score),
          'auc':make_scorer(roc_auc_score)}

kfold = StratifiedKFold(n_splits=3, shuffle=True)
clf = GridSearchCV(xgb_model, parameters,n_jobs=-1,
                   cv=kfold, 
                   scoring=scoring,
                   verbose=3, refit='auc')
clf.fit(X_train,y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 137.8min finished
